In [0]:
import nltk

In [33]:
with open('/content/drive/My Drive/Data/HungerGamex.txt', 'r') as myFile:
  text = myFile.read()

chars = sorted(list(set(text)))
char_indices = dict((c, i)
  for i, c in enumerate(chars))
indicies_char = dict((i, c)
  for i, c in enumerate(chars))
'corpus length: {} total chars: {}'.format(len(text), len(chars))

'corpus length: 545112 total chars: 81'

In [34]:
print(text[:500])

For James Proimos 



PARTI 

THE TRIBUTES 



When I wake up, the other side of the bed is cold. My fin¬ 
gers stretch out, seeking Prim's warmth but finding only the 
rough canvas cover of the mattress. She must have had bad 
dreams and climbed in with our mother. Of course, she did. 
This is the day of the reaping. 

I prop myself up on one elbow. There's enough light in the 
bedroom to see them. My little sister, Prim, curled up on her 
side, cocooned in my mother's body, their cheeks presse


In [35]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range (0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 181691


In [36]:
print(sentences[0:20])

['For James Proimos \n\n\n\nPARTI \n\nTHE TRIBUT', ' James Proimos \n\n\n\nPARTI \n\nTHE TRIBUTES ', 'mes Proimos \n\n\n\nPARTI \n\nTHE TRIBUTES \n\n\n', ' Proimos \n\n\n\nPARTI \n\nTHE TRIBUTES \n\n\n\nWh', 'oimos \n\n\n\nPARTI \n\nTHE TRIBUTES \n\n\n\nWhen ', 'os \n\n\n\nPARTI \n\nTHE TRIBUTES \n\n\n\nWhen I w', '\n\n\n\nPARTI \n\nTHE TRIBUTES \n\n\n\nWhen I wake', '\nPARTI \n\nTHE TRIBUTES \n\n\n\nWhen I wake up', 'RTI \n\nTHE TRIBUTES \n\n\n\nWhen I wake up, t', ' \n\nTHE TRIBUTES \n\n\n\nWhen I wake up, the ', 'THE TRIBUTES \n\n\n\nWhen I wake up, the oth', ' TRIBUTES \n\n\n\nWhen I wake up, the other ', 'IBUTES \n\n\n\nWhen I wake up, the other sid', 'TES \n\n\n\nWhen I wake up, the other side o', ' \n\n\n\nWhen I wake up, the other side of t', '\n\nWhen I wake up, the other side of the ', 'hen I wake up, the other side of the bed', ' I wake up, the other side of the bed is', 'wake up, the other side of the bed is co', 'e up, the other side of the bed is cold.']


In [0]:
import numpy as np

X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    X[i, t, char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1

In [0]:
%tensorflow_version 2.x

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [EarlyStopping(monitor="loss", patience=2),
            ModelCheckpoint(filepath="bestmodel.h5", monitor="loss", save_best_only=True)]

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               107520    
_________________________________________________________________
dense_2 (Dense)              (None, 81)                10449     
_________________________________________________________________
activation_2 (Activation)    (None, 81)                0         
Total params: 117,969
Trainable params: 117,969
Non-trainable params: 0
_________________________________________________________________


In [47]:
epochs = 40
batch_size = 128
model.fit(X, y, batch_size=batch_size, epochs=epochs, callbacks=callbacks)
model.save_weights("/content/drive/My Drive/Data/lotr_weights/lotr_lstm_weights.h5")

Epoch 1/40
1420/1420 [==============================] - 10s 7ms/step - loss: 1.2778
Epoch 2/40
1420/1420 [==============================] - 10s 7ms/step - loss: 1.2751
Epoch 3/40
1420/1420 [==============================] - 10s 7ms/step - loss: 1.2735
Epoch 4/40
1420/1420 [==============================] - 10s 7ms/step - loss: 1.2689
Epoch 5/40
1420/1420 [==============================] - 10s 7ms/step - loss: 1.2701
Epoch 6/40
1420/1420 [==============================] - 10s 7ms/step - loss: 1.2667
Epoch 7/40
1420/1420 [==============================] - 10s 7ms/step - loss: 1.2650
Epoch 8/40
1420/1420 [==============================] - 8s 6ms/step - loss: 1.2630
Epoch 9/40
1420/1420 [==============================] - 8s 6ms/step - loss: 1.2616
Epoch 10/40
1420/1420 [==============================] - 9s 6ms/step - loss: 1.2605
Epoch 11/40
1420/1420 [==============================] - 9s 7ms/step - loss: 1.2590
Epoch 12/40
1420/1420 [==============================] - 9s 6ms/step - loss: 1

In [0]:
import random 

def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [50]:
import sys 

start_index = random.randint(0, len(text) - maxlen - 1)

for diversity in [0.2, 0.5, 1.0]:
  print()
  print('-----diversity:', diversity)
  generated = ''
  sentence = text[start_index: start_index + maxlen]
  generated += sentence
  print('------Generated with seed: "' + sentence + '"')
  sys.stdout.write(generated)
  for i in range(400):
    x = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      x[0, t, char_indices[char]] = 1.
    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, diversity)
    next_char = indicies_char[next_index]
    generated += next_char
    sentence = sentence[1:] + next_char
    sys.stdout.write(next_char)
    sys.stdout.flush()
  print()


-----diversity: 0.2
------Generated with seed: "a relief to be alone with Cinna, to feel"
a relief to be alone with Cinna, to feel a sort of the 
pair of the 
supplies to c

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


an see the 
stinges and seems to the stream 
with the stream and seems to care the 
part of the 
legs and had and I can't stare a stream and the pack and 
she's a straight and the stream and the stream and then well the stream and find a straight of the 
part of the 
pair of the 
stinges and he and I think the 
same something and 
she's a stream and 
she's

-----diversity: 0.5
------Generated with seed: "a relief to be alone with Cinna, to feel"
a relief to be alone with Cinna, to feel better the boy from District 
ing 
about the rain. The same sleeping bag with my 
father and saddep the cheese stares the same lose it is so as I spread and he was a suppled out a but the 
Games berow me and I didn't see the stinged in the 
train and the way 
the hard are have to guess is sense. 

I can see the boy from District 
ided the 
room. Someone if I always and the train has been a few st

-----diversity: 1.0
------Generated with seed: "a relief to be alone with Cinna, to feel"
a relief to be alo